In [19]:
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz
from numpy import save
import itertools

In [20]:
ndc = pd.read_csv("../data/working/ndc_clean.csv", index_col = [0])

In [21]:
ndc.dropna(inplace = True)
ndc.reset_index(inplace = True)

In [22]:
ndc_row = ndc['row']

In [23]:
ndc_org = ndc['original_company']

In [24]:
ndc_clean = ndc['cleaned_name']

In [25]:
dna = pd.read_csv("../data/working/dna_clean.csv", index_col = [0])

In [26]:
dna.dropna(inplace = True)
dna.reset_index(inplace = True)

In [11]:
dna.columns

Index(['index', 'Unnamed: 0.1', 'Code', 'Description', 'cleaned_companies'], dtype='object')

In [27]:
dna_row = dna['index']

In [28]:
dna_org = dna['Description']

In [29]:
dna_clean = dna['cleaned_companies']

### Matching across NDC and DNA

In [30]:
##Function that takes in 3 lists, the list of strings you want to compare, the list of strings you're comparing to and a list with the original way the string from list 1 was written.  
def matchWithFuzzyNamesOG(l1, l2, og1, og2, row_num1, row_num2):
    match = pd.DataFrame(columns = ['clean NDC company', 'original NDC company', 'NDC row', 'clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio'])
    for (i, o, r) in zip(l1, og1, row_num1):
        matches = []
        score = []
        #score_partial = []
        row = []
        matchesog = []
        for (j, q, s) in zip(l2, og2, row_num2):
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                matchesog.append(q)
                score.append(fuzz.ratio(i,j))
                row.append(s)
                #the partial ratio will return the ratio raw score of 100  if the shorter string is found an any of substrings of the larger string, used this as a safety net and potentially as an additional step to 
               # score_partial.append((fuzz.partial_ratio(i,j)))
                match = match.append({ 'clean NDC company': i, 'original NDC company': o, 'NDC row': r, 'clean DNA row': row, 'clean DNA company': matches,   'original DNA company': matchesog, 'fuzz ratio': score,},ignore_index=True)
        
#Will output a pandas dataframe with 4 columns
    return match

In [31]:
ndcxdna_match = matchWithFuzzyNamesOG(ndc_clean, dna_clean, ndc_org, dna_org, ndc_row, dna_row)

In [15]:
ndcxdna_match

,clean NDC company,original NDC company,NDC row,clean DNA row,clean DNA company,original DNA company,fuzz ratio
0,cardinal,.Cardinal Health,2,"[9896, 10970, 11719, 12082, 18921, 32394]","[cardinal, cardinal, cardinal, cardinal, cardi...","[Cardinal Partners, Cardinal Health Inc, Cardi...","[100, 100, 100, 100, 100, 100]"
1,cardinal,.Cardinal Health,2,"[9896, 10970, 11719, 12082, 18921, 32394]","[cardinal, cardinal, cardinal, cardinal, cardi...","[Cardinal Partners, Cardinal Health Inc, Cardi...","[100, 100, 100, 100, 100, 100]"
2,cardinal,.Cardinal Health,2,"[9896, 10970, 11719, 12082, 18921, 32394]","[cardinal, cardinal, cardinal, cardinal, cardi...","[Cardinal Partners, Cardinal Health Inc, Cardi...","[100, 100, 100, 100, 100, 100]"
3,cardinal,.Cardinal Health,2,"[9896, 10970, 11719, 12082, 18921, 32394]","[cardinal, cardinal, cardinal, cardinal, cardi...","[Cardinal Partners, Cardinal Health Inc, Cardi...","[100, 100, 100, 100, 100, 100]"
4,cardinal,.Cardinal Health,2,"[9896, 10970, 11719, 12082, 18921, 32394]","[cardinal, cardinal, cardinal, cardinal, cardi...","[Cardinal Partners, Cardinal Health Inc, Cardi...","[100, 100, 100, 100, 100, 100]"
...,...,...,...,...,...,...,...
1923,zee,Zee Medical,6976,[63599],[zee],[Zee Medical Inc],[100]
1924,zee,Zee Medical Inc,6977,[63599],[zee],[Zee Medical Inc],[100]
1925,zep,Zep Inc.,6983,[63637],[zep],[Zep Inc.],[100]
1926,zep,"Zep, Inc",6984,[63637],[zep],[Zep Inc.],[100]


In [32]:
ndcxdna_match = ndcxdna_match[~ndcxdna_match.duplicated('NDC row')]

In [33]:
ndcxdna_match

,clean NDC company,original NDC company,NDC row,clean DNA row,clean DNA company,original DNA company,fuzz ratio
0,cardinal,.Cardinal Health,2,"[9896, 10970, 11719, 12082, 18921, 32394]","[cardinal, cardinal, cardinal, cardinal, cardi...","[Cardinal Partners, Cardinal Health Inc, Cardi...","[100, 100, 100, 100, 100, 100]"
6,xl,2xl Corporation,14,[63030],[xl],[XL Group Ltd],[100]
7,innovations,323 Innovations,15,[1920],[innovations],[American Innovations Ltd.],[100]
8,lab,3LAB,17,[34643],[lab],[Lab21 Ltd],[100]
9,lab,"3LAB, Inc",18,[34643],[lab],[Lab21 Ltd],[100]
...,...,...,...,...,...,...,...
1923,zee,Zee Medical,6976,[63599],[zee],[Zee Medical Inc],[100]
1924,zee,Zee Medical Inc,6977,[63599],[zee],[Zee Medical Inc],[100]
1925,zep,Zep Inc.,6983,[63637],[zep],[Zep Inc.],[100]
1926,zep,"Zep, Inc",6984,[63637],[zep],[Zep Inc.],[100]


In [34]:
ndcxdna_match['clean NDC company'] = ndcxdna_match['clean NDC company'].astype(str)
ndcxdna_match['original NDC company'] = ndcxdna_match['original NDC company'].astype(str)
ndcxdna_match['NDC row'] = ndcxdna_match['NDC row'].astype(str)

In [35]:
ndcxdna_match_unique = ndcxdna_match.groupby('clean NDC company', as_index = False).agg({'original NDC company': ', '.join, 
                                                                            'NDC row': ', '.join
                                                                                })

In [36]:
ndcxdna_match_unique

,clean NDC company,original NDC company,NDC row
0,-,65364-110,29
1,-eleven,"7-Eleven, 7-Eleven Inc, 7-Eleven Inc., 7-Eleve...","31, 32, 33, 34"
2,a,"A19, LLC",54
3,abbvie,AbbVie Inc.,66
4,able c&c,"Able C&C Co., Ltd.",70
...,...,...,...
795,yes to,"Yes To Inc., Yes To Incorporated, Yes To, Inc.","6942, 6943, 6944"
796,young living essential oils,"Young Living Essential Oils, LC",6955
797,zee,"Zee Medical, Zee Medical Inc","6976, 6977"
798,zep,"Zep Inc., Zep, Inc","6983, 6984"


In [37]:
ndcxdna_match1 = ndcxdna_match.groupby('clean NDC company', as_index = False).first()

In [38]:
ndcxdna_match_unique[['corporate family', 'clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio']] = ndcxdna_match1[['clean NDC company','clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio']]

In [39]:
ndcxdna_match_unique.to_csv('../data/working/ndc_dna_matching.csv')